In [49]:
import pandas as pd
import numpy as np
df = pd.read_csv('rede_fraktion_preprocessed.csv')
tops = df['top'].unique()

In [50]:
df['top_id'] = df['top'].astype('category').cat.codes

In [51]:
df['top_id'].max()

1850

In [47]:
import re
import numpy as np
df['top'] = df['top'].apply(lambda x: re.sub(r'Drucksache ([0-9]+)/([0-9]+)', '', x))
df['top'] = df['top'].apply(lambda x: re.sub(r'(Befragung der Bundesregierung|Befragung|Fragestunde)', '', x))
df_tops = df[['top', 'top_id']].drop_duplicates(subset=['top', 'top_id'])
df_tops.index = np.arange(df_tops.shape[0])
df_tops_to_ids = df[['top_id']]

In [48]:
df_tops_to_ids.to_csv('top_to_id ')

,top_id
0,237
1,237
2,237
3,237
4,237
...,...
279483,861
279484,861
279485,861
279486,861


In [4]:
!python -m spacy download de_core_news_md

2021-08-11 16:31:23.053577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-11 16:31:23.053614: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47.8 MB 108 kB/s eta 0:00:01    |▎                               | 430 kB 2.5 MB/s eta 0:00:19
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')


In [5]:
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

nlp = spacy.load('de_core_news_md')
german_stop_words = stopwords.words('german')

def stop_word_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])

def lemmatization(x):
    doc = nlp(x)
    return ' '.join([x.lemma_ for x in doc])

col = pd.Series(tops).apply(lambda x: stop_word_removal(x))

processed_tops = []
for x in tqdm(col):
    processed_tops.append(lemmatization(x))
    
indexes = []
result = []
for i, x in tqdm(enumerate(processed_tops)):
    tok = word_tokenize(x, language='german')
    if len(tok) > 3:
        indexes.append(i)
        result.append(" ".join(tok))

  0%|          | 0/1776 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [6]:
import joblib
joblib.dump({
    'tops': tops[indexes],
    'processed': result
}, 'tops_processed.pkl')

['tops_processed.pkl']

In [13]:
import joblib
data = joblib.load('tops_processed.pkl')
tops = data['tops']

In [14]:
import numpy as np
tops = np.array(tops)

In [15]:
import tensorflow_hub as hub
import tensorflow_text
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [16]:
enc = embed(tops).numpy()

In [17]:
%matplotlib widget
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
XX = PCA(n_components=2).fit_transform(enc)
sns.scatterplot(x=XX[:, 0], y=XX[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [18]:
%matplotlib widget
import numpy as np
from sklearn.cluster import AgglomerativeClustering
c = AgglomerativeClustering(n_clusters=None, distance_threshold=0.75, affinity='cosine', linkage='average').fit_predict(enc)
print(np.unique(c))

XX = PCA(n_components=2).fit_transform(enc)
plt.scatter(*XX.T)
sns.scatterplot(x=XX[:, 0], y=XX[:, 1], hue=c)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [23]:
tops[c == 22]

array(['Unabhängige Polizeibeschwerdestelle auf Bundesebene einrichten',
       'Für ein Recht auf Anonymität im öffentlichen Raum – Keine automatisierte Gesichtserkennung durch die Bundespolizei Freiheit und Rechtsstaatlichkeit erhalten – Kein Einsatz biometrischer Gesichtserkennung in öffentlichen Räumen',
       'Erfassung von Straftaten unter Zuhilfenahme des Tatmittels Messer in der Polizeilichen Kriminalstatistik',
       ' Für ein Waffengesetz mit Augenmaß – Kein Generalverdacht gegen legale Waffenbesitzer Freiräume für Jäger und Sportschützen – Für eine schonende Umsetzung der EU-Feuerwaffenrichtlinie Tödliche Gefahr durch Schusswaffen eindämmen',
       'Aktuelle Stunde Kommunalpolitiker, Polizei und Rettungskräfte vor Drohungen und Gewalt wirksam schützen',
       'Aktuelle Stunde Erfolge bei der Bekämpfung der Kriminalität – Zahlen der Polizeilichen Kriminalstatistik 2018',
       ' Für ein Recht auf Anonymität im öffentlichen Raum – Keine automatisierte Gesichtserkennung du

In [24]:
def cosine_similarity(vector_1, vector_2):
    return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1) * np.linalg.norm(vector_2))

def nearest_neighbor(v, candidates, k=1):
    indices = np.argsort([cosine_similarity(v, row) for row in candidates])[-k:]
    distances = [cosine_similarity(v, row) for row in candidates[indices]]
    return indices, distances

In [28]:
%matplotlib widget
sentence = 'Kohleausstieg'
e = embed([sentence]).numpy()[0]
neighbor_indx, distances = nearest_neighbor(e, enc, k=30)
print(tops[neighbor_indx])
plt.plot(distances)

['Auswirkungen der Afrikanischen Schweinepest auf die Agrar- und Ernährungswirtschaft'
 'Nord Stream 2 die politische Unterstützung entziehen und den Bau stoppen Energiesicherheit gewährleisten – Nord Stream 2 unterstützen'
 'Nachhaltige Entwicklungsziele erreichen – Potenziale aus der Agrarökologie anerkennen und unterstützen'
 'Klimaziele verantwortungsbewusst erreichen Klimaziel 2020 einhalten statt verschieben – Zwanzig älteste Braunkohlekraftwerke unverzüglich abschalten Klimakonferenz in Bonn – Schneller Ausstieg aus der Kohle ist jetzt nötig Klimaschutzzusagen einhalten – An Zielen für 2020 festhalten Ausbau der Windenergie sichern, Klimaschutz voranbringen und Standort für Zukunftstechnologien erhalten'
 'Bürgerenergie retten Stromstau auflösen anstatt erneuerbare Energien zu bremsen'
 'Tschernobyl mahnt – Atomausstieg konsequent umsetzen'
 'Wirksames Klimaschutzgesetz vorlegen – Maßnahmen und Regelungen für alle Sektoren UN-Klimakonferenz in Katowice 2018 – Pariser Klimaabkomm

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Load saved BERT (huBERTusHeil)

In [ ]:
import torch.nn as nn
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from tqdm.notebook import tqdm

PRE_TRAINED_MODEL_NAME = 'bert-base-german-cased'

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = self.bert.config.hidden_size, 25, 7

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

    def forward_latent(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        return self.classifier[0](last_hidden_state_cls)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_classifier = BertClassifier(freeze_bert=False)
bert_classifier.load_state_dict(torch.load("huBERTusHeil/20210809-1337-4epochs.pt", map_location=torch.device(device)))
bert_classifier.to(device)

# Preprocessing

In [ ]:
MAX_LEN = 512

def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for i, d in tqdm(enumerate(data), total=data.shape[0]):
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        #print(f'processing sample {i} of {len(data)}')
        encoded = tokenizer.encode_plus(
            text=d,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded.get('input_ids'))
        attention_masks.append(encoded.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

def get_latent_vector(model, input_ids, attention_mask):
    # Feed input to BERT
    outputs = model.bert(input_ids=input_ids,
                        attention_mask=attention_mask)

    # Extract the last hidden state of the token `[CLS]` for classification task
    last_hidden_state_cls = outputs[0][:, 0, :]

    # Feed input to first layer of classifier
    return model.classifier[0](last_hidden_state_cls)
def evaluate_latent_space(model, dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    speeches = []
    y_true = []

    # For each batch in our validation set...
    for batch in tqdm(dataloader):
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            vec = get_latent_vector(model, b_input_ids, b_attn_mask)

        speeches.append(vec.cpu().numpy())
        
    # Compute the average accuracy and loss over the validation set.
    return np.concatenate(speeches)

## Model Evaluation

## Test Out the Model

In [ ]:
import seaborn as sns
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.decomposition import PCA
import seaborn as sns

batch_size = 32

fractions = ["AfD", "B90", "Union", "Linke", "FDP", "SPD", "fraktionslos"]
m = {x: i+1 for i, x in enumerate(fractions)}

# tokenize and evaluate all speeches in partial_df
def show_latent_space(partial_df):
    input_ids, attention_masks = preprocessing_for_bert(partial_df['text'])
    dataset = TensorDataset(input_ids, attention_masks)
    data_sampler = SequentialSampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    
    latent = evaluate_latent_space(bert_classifier, dataloader)
    labels = partial_df['fraktion'].map(m)
    return latent, labels

In [ ]:
%matplotlib widget
latent, labels = show_latent_space(df[:2000])
XX = PCA(n_components=2).fit_transform(latent)
palette = ["#0000FF", "#00FF00", "#000000", "#EEEE00",  "#FF00FF", "#EE0000", "#CCCCCC"]
sns.scatterplot(x=XX[:, 0], y=XX[:, 1], hue=labels, palette=palette[:max(labels)])

In [ ]:
%matplotlib widget
XX = PCA(n_components=2).fit_transform(latent)
palette = ["#0000FF", "#00FF00", "#000000", "#FF00FF", "#EEEE00", "#EE0000", "#CCCCCC"]
sns.scatterplot(x=XX[:, 0], y=XX[:, 1], hue=labels, palette=palette[:max(labels)])